## Load Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## Define Transformations

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

## Load the Data

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_data = datasets.CIFAR10( root="./data", train=True, download=True, transform = transform)
test_data = datasets.CIFAR10( root="./data", train=False, download=True, transform = transform)

In [4]:
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

## Build the Architecture

In [24]:
class cnn_cifar(nn.Module):
    def __init__(self):
        super(cnn_cifar, self).__init__()
        self.cnn1 = nn.Conv2d(3, 6, 5)
        self.cnn2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x=self.cnn1(x)
        x=torch.relu(x)
        x=torch.max_pool2d(x, 2)

        x=self.cnn2(x)
        x=torch.relu(x)
        x=torch.max_pool2d(x, 2)

        x=x.flatten(1)

        x=self.fc1(x)
        x=torch.relu(x)

        x=self.fc2(x)
        x=torch.relu(x)

        x=self.fc3(x)

        return torch.relu(x)

In [26]:
model = cnn_cifar()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

##  Training

In [28]:
for epoch in range(10):
    for images, label in train_loader:
        outputs = model(images)
        loss = criterion(outputs, label)
        optimizer.step()
        optimizer.zero_grad()
        

epoch : 0 ---- loss : 2.311457633972168
epoch : 0 ---- loss : 2.3072001934051514
epoch : 0 ---- loss : 2.30446195602417
epoch : 0 ---- loss : 2.303915023803711
epoch : 0 ---- loss : 2.310959577560425
epoch : 0 ---- loss : 2.2953457832336426
epoch : 0 ---- loss : 2.298208475112915
epoch : 0 ---- loss : 2.3070662021636963
epoch : 0 ---- loss : 2.3045544624328613
epoch : 0 ---- loss : 2.310750722885132
epoch : 0 ---- loss : 2.3024966716766357
epoch : 0 ---- loss : 2.3083341121673584
epoch : 0 ---- loss : 2.3035645484924316
epoch : 0 ---- loss : 2.3065309524536133
epoch : 0 ---- loss : 2.306830406188965
epoch : 0 ---- loss : 2.3006391525268555
epoch : 0 ---- loss : 2.3030545711517334
epoch : 0 ---- loss : 2.2952423095703125
epoch : 0 ---- loss : 2.3088293075561523
epoch : 0 ---- loss : 2.3053765296936035
epoch : 0 ---- loss : 2.307609796524048
epoch : 0 ---- loss : 2.305352210998535
epoch : 0 ---- loss : 2.298738956451416
epoch : 0 ---- loss : 2.3031904697418213
epoch : 0 ---- loss : 2.311

KeyboardInterrupt: 